In [8]:
import os
import random
filepath = '/Users/Peter/Desktop/5000/Data Science/Projects/Python/MusicBox/data/data/data/'

In [4]:
FileName = ['play/all_play.log', 'search/all_search.log', 'down/all_down.log']
date_time = ['20170505', '20170428', '20170421', '20170414', '20170407', '20170331']
dict_list = [set(), set(), set(), set(), set(), set()]
for name in FileName:
    print('File %s Start' % name)
    with open(filepath + name, 'r') as f:
        for line in f:
            line_list = line[:-1].split('\t')
            for index in range(len(date_time)):
                if line_list[2] > date_time[index]:
                    dict_list[index].add(line_list[0])
                    break

File play/all_play.log Start
File search/all_search.log Start
File down/all_down.log Start


In [5]:
for dictionary in dict_list:
    print('Active Users: %s' % len(dictionary))

Active Users: 177190
Active Users: 190713
Active Users: 202442
Active Users: 229237
Active Users: 256258
Active Users: 352023


In [6]:
import matplotlib.pyplot as plt
plt.plot([len(dictionary) for dictionary in dict_list])

In [6]:
stay_rate = [0] * 6
turnover_rate = [0] * 6
for index in range(1, len(dict_list)):
    same = (dict_list[index-1] & dict_list[index])
    stay_rate[index] = round(float(len(same)) / float(len(dict_list[index-1])), 5)
    if (len(dict_list[index - 1]) - len(same) != 0):
        turnover_rate[index] = round(float(len(dict_list[index]) - len(same))/float(len(dict_list[index - 1]) - len(same)), 5)

In [13]:
print('Stay Rate is: %s' % stay_rate)
print('TurnOver Rate (leave : come) is: %s' % turnover_rate)

Stay Rate is: [0, 0.75611, 0.74774, 0.77356, 0.77067, 0.82245]
TurnOver Rate (leave : come) is: [0, 1.31292, 1.2438, 1.58453, 1.514, 3.10477]


In [14]:
FixedDatePoint = '20170421'

In [20]:
active_users = dict()
with open(filepath + 'play/all_play.log', 'r') as f:
    for line in f:
        line_list = line.split('\t')
        line_list = [line.replace(" ", "") for line in line_list]
        if (len(line_list) == 3):
            if line_list[2] < FixedDatePoint and line_list[2] >= '20170330':
                if line_list[0] in active_users:
                    active_users[line_list[0]] += 1
                else:
                    active_users[line_list[0]] = 1
Active_Users_Before = set()
for user, count in active_users.items():
    if count > 3:
        Active_Users_Before.add(user)

Active Users are defined as users who plays songs for more than 3 times.

active User Number after and before simple filtering (3 times play records)

In [27]:
print(len(Active_Users_Before))
print(len(active_users))

477300
568503


In [28]:
Active_Users_After = set()

for name in FileName:
    print('File %s Start' % name)
    with open(filepath + name, 'r') as f:
        for line in f:
            line_list = line.split('\t')
            line_list = [line.replace(" ", "") for line in line_list]
            if (len(line_list) == 3):
                if line_list[2] >= FixedDatePoint:
                    Active_Users_After.add(line_list[0])
    print('File %s Complete' % name)

churnRate = 0.5
activeRate = 0.5
churnUser = set()
for user in Active_Users_Before:
    if (user not in Active_Users_After):
        churnUser.add(user)
print('active users before is: %d' % len(Active_Users_Before))
print('churnUser Number is: %d' % len(churnUser))
chosenUser = set()
churnUserNum = 0
activeUserNum = 0
for user in Active_Users_Before:
    if (user in churnUser):
        if random.random() < churnRate:
            chosenUser.add(user)
            churnUserNum += 1
    else:
        if random.random() < activeRate:
            chosenUser.add(user)
            activeUserNum += 1
print('chosenUser Number is: %d' % len(chosenUser))
print('Active User Before Number is: %d' % len(Active_Users_Before))
print('Active User After Number is: %d' % len(Active_Users_After))
print('Active Chosen User Number is :%d' % activeUserNum)
print('Churn Chosen User Number is :%d' % churnUserNum)

File play/all_play.log Start
File play/all_play.log Complete
File search/all_search.log Start
File search/all_search.log Complete
File down/all_down.log Start
File down/all_down.log Complete
active users before is: 477300
churnUser Number is: 236222
chosenUser Number is: 238828
Active User Before Number is: 477300
Active User After Number is: 282575
Active Chosen User Number is :120724
Churn Chosen User Number is :118104


In [29]:
outputFile = open('select.log', 'w')
typeName = ['p', 's', 'd']
index = 0
for name in FileName:
    print('File %s Output Start' % name)
    with open(filepath + name, 'r') as f:
        for line in f:
            line = line[:-1].split('\t')
            if (len(line) == 3 and line[0] in chosenUser):
                line = line[0] + '\t' + line[1] + '\t' + line[2] + '\t' +  typeName[index] + '\n'
                outputFile.write(line)
    print('File %s Output Complete' % name)
    index += 1

outputFile.close()

File play/all_play.log Output Start
File play/all_play.log Output Complete
File search/all_search.log Output Start
File search/all_search.log Output Complete
File down/all_down.log Output Start
File down/all_down.log Output Complete


In [31]:
import pandas as pd
from datetime import datetime

Fixed_Date = '20170421'
user_info = dict()
type_dict = {'s': 0, 'p': 1, 'd':2}
date = ['20170421', '20170415', '20170408']
with open('select.log', 'r') as f:
    for line in f:
        line = line[:-1].split('\t')
        if line[0] not in user_info:
            user_info[line[0]] = [line[1]] + [0] * 9 + [1]
        if (line[2] > Fixed_Date):
            user_info[line[0]][10] = 0
        else:
            type_index = type_dict[line[3]]
            for i in range(len(date)):
                if (line[2] >= date[i]):
                    user_info[line[0]][type_index * 3 + 1 + i] += 1
        

print('user info generate complete')
names = ['user_id', 'phone', 'search_1', 'search_7', 'search_14', 
         'play_1', 'play_7', 'play_14', 'down_1', 'down_7', 'down_14', 'churn']
data = dict()
for name in names:
    data[name] = []

for user_id, info in user_info.items():
    data[names[0]].append(user_id)
    for i in range(len(info)):
        data[names[i + 1]].append(info[i])
print('user info data frame generate complete')

df = pd.DataFrame(data = data)

df.to_csv('generate.csv', index = False)
print('user info dataframe output complete')
df.to_csv('generate.csv', index = False)

user info generate complete
user info data frame generate complete
user info dataframe output complete


In [32]:
df_generate = pd.read_csv('generate.csv')

In [34]:
df_generate = df_generate[names]

In [35]:
df_generate.head(10)

,user_id,phone,search_1,search_7,search_14,play_1,play_7,play_14,down_1,down_7,down_14,churn
0,167591585,ar,0,0,0,0,0,0,0,0,0,1
1,168134533,ar,0,4,10,22,36,70,0,0,1,0
2,167891787,ar,0,0,0,28,44,71,0,0,0,0
3,167591586,ar,0,0,0,0,66,147,0,0,0,1
4,168113743,ar,0,3,8,0,7,13,0,0,0,0
5,167591582,ar,0,0,0,0,0,66,0,0,0,1
6,168743941,ar,0,0,0,23,279,484,0,0,0,0
7,168623764,ar,0,0,0,0,0,0,0,0,0,1
8,167910301,ar,0,0,0,8,25,75,0,0,0,0
9,167910302,ip,0,0,0,22,199,310,0,0,0,0


In [36]:
df_generate.to_csv('generate.csv', index = False)

In [9]:
import pandas as pd
df_generate = pd.read_csv('generate.csv')
df_generate.head(10)

,user_id,phone,search_1,search_7,search_14,play_1,play_7,play_14,down_1,down_7,down_14,churn
0,167591585,ar,0,0,0,0,0,0,0,0,0,1
1,168134533,ar,0,4,10,22,36,70,0,0,1,0
2,167891787,ar,0,0,0,28,44,71,0,0,0,0
3,167591586,ar,0,0,0,0,66,147,0,0,0,1
4,168113743,ar,0,3,8,0,7,13,0,0,0,0
5,167591582,ar,0,0,0,0,0,66,0,0,0,1
6,168743941,ar,0,0,0,23,279,484,0,0,0,0
7,168623764,ar,0,0,0,0,0,0,0,0,0,1
8,167910301,ar,0,0,0,8,25,75,0,0,0,0
9,167910302,ip,0,0,0,22,199,310,0,0,0,0


In [10]:
df_generate.shape

(238828, 12)